Modelación de AgentPy

In [1]:
import agentpy as ap
import numpy as np
import matplotlib.pyplot as plt
import math
import IPython
import json
import random

In [2]:
class Semaphore(ap.Agent):
    def setup(self):
        self.step_time = 0.1
        self.direction = [0,1]
        self.state = 0 # verde = 0, amarillo = 1, rojo = 2
        self.state_time = 0
        
        self.green_duration = 90
        self.yellow_duration = 5
        self.red_duration = 60
        
    def update(self):
        self.state_time += self.step_time
        
        #verde a amarillo
        if self.state == 0:
            if self.state_time >= self.green_duration:
                self.state = 1
                self.state_time = 0
        #amarillo a rojo        
        if self.state == 1:
            if self.state_time >= self.yellow_duration:
                self.state = 2
                self.state_time = 0
        #rojo a verde      
        if self.state == 2:
            if self.state_time >= self.red_duration:
                self.state = 0
                self.state_time = 0
        
    def set_green(self):
        self.state = 0
        self.state_time = 0
    
    def set_yellow(self):
        self.state = 1
        self.state_time = 0
        
    def set_red(self):
        self.state = 2
        self.state_time = 0

In [3]:
class Car(ap.Agent):
    
    def setup(self):
        self.step_time = 0.1
        self.direction = [1,0]
        self.speed = 0.0
        self.max_speed = 16.66
        self.state = 1 # 0 = chocado, 1 = ok
        
    def update_position(self):
        if self.state == 0:
            return 
        
        self.model.avenue.move_by(self,[self.speed*self.step_time*self.direction[0],self.speed*self.step_time*self.direction[1]])
        
    def update_speed(self):
        if self.state == 0:
            return
        
        p = self.model.avenue.positions[self]
        
        min_car_distance = 1000000
        
        for car in self.model.cars:
            if car != self:
                
                dot_p1 = self.direction[0]*car.direction[0] + self.direction[1]*car.direction[1]
            
                p2 = self.model.avenue.positions[car]
            
                dot_p2 = (p2[0]-p[0])*self.direction[0] + (p2[1]-p[1])*self.direction[1]
            
                if dot_p1 > 0 and dot_p2 > 0:
                    d = math.sqrt((p[0]-p2[0])**2 + (p[1]-p2[1])**2)
                    
                    if d < min_car_distance:
                        min_car_distance = d
        
        min_semaphore_distance = 1000000
        semaphore_state = 0
        for semaphore in self.model.semaphores:
                
            dot_p1 = self.direction[0]*semaphore.direction[0] + self.direction[1]*semaphore.direction[1]
            
            p2 = self.model.avenue.positions[semaphore]
            
            dot_p2 = (p2[0]-p[0])*self.direction[0] + (p2[1]-p[1])*self.direction[1]
            
            if dot_p1 < 0 and dot_p2 > 0:
                d = math.sqrt((p2[0]-p[0])**2 + (p2[1]-p[1])**2)
                
                if d < min_semaphore_distance:
                    min_semaphore_distance = d
                    semaphore_state = semaphore.state
            
        if min_car_distance < 2:
            self.speed = 0
            self.state = 1
            
        elif min_car_distance < 13:
            self.speed = np.maximum(self.speed - 15*self.step_time, 0)
        
        elif min_car_distance < 20:
            self.speed = np.maximum(self.speed - 10*self.step_time, 0)
            
        elif min_semaphore_distance < 20 and semaphore_state == 1:
            self.speed = np.minimum(self.speed + 2.5*self.step_time, self.max_speed)
            
        elif min_semaphore_distance < 40 and semaphore_state == 1:
            self.speed = np.maximum(self.speed - 2.5*self.step_time, 5)
            
        elif min_semaphore_distance < 30 and semaphore_state == 2:
            self.speed = np.maximum(self.speed - 10*self.step_time, 0)
        
        elif min_semaphore_distance < 50 and semaphore_state == 2:
            self.speed = np.maximum(self.speed - 2.5*self.step_time, 5)
        
        else:
            self.speed = np.minimum(self.speed + 2.0*self.step_time, self.max_speed)
        
        
        

In [4]:
class AvenueModel(ap.Model):
    
    def setup(self):
        self.cars = ap.AgentList(self, self.p.cars, Car)
        self.cars.step_time = self.p.step_time
        self.cars.speed = self.p.v0
        
        c_north = int(self.p.cars/2)
        c_south = self.p.cars - c_north
        
        for k in range(c_north):
            self.cars[k].direction = [0,1]
            
        for k in range(c_south):
            self.cars[c_north + k].direction = [0,-1]
            
        self.semaphores = ap.AgentList(self, 2, Semaphore)
        self.semaphores.step_time = self.p.step_time
        self.semaphores.green_duration = self.p.green
        self.semaphores.yellow_duration = self.p.yellow
        self.semaphores.red_duration = self.p.red
        self.semaphores[0].direction = [0,1]
        self.semaphores[1].direction = [0,-1]
        
        self.avenue = ap.Space(self, shape = [60,self.p.size], torus = True)
        
        self.avenue.add_agents (self.semaphores, random = True)
        self.avenue.move_to(self.semaphores[0],[10, self.p.size*0.5 - 5])
        self.avenue.move_to(self.semaphores[1],[50, self.p.size*0.5 + 5])
        
        self.avenue.add_agents(self.cars, random = True)
        
        for k in range(c_north):
            self.avenue.move_to(self.cars[k],[34,10*(k+1)])
            
        for k in range(c_south):
            self.avenue.move_to(self.cars[k + c_north],[26,self.p.size - 10*(k+1)])
        self.frames = 0

        # Archivo json
        self.data = {}
        self.data['size'] = []
        self.data['size'].append({'size': self.p.size})
        self.data['cars'] = []
        self.data['semaphores'] = []
        self.data['frames'] = []

        # Carga de carros
        for k in range(self.p.cars):
            self.data['cars'].append({
                'id': k,
                'x': self.model.avenue.positions[self.cars[k]][0],
                'z': self.model.avenue.positions[self.cars[k]][1],
                'dir': math.atan2(self.cars[k].direction[0], self.cars[k].direction[1]) * 180 / math.pi,
                'type': random.randint(0, 6)
            })

        # Carga de semaforos
        for k in range(len(self.semaphores)):
            self.data['semaphores'].append({
                'id': k + model.p.cars,
                'x': self.model.avenue.positions[self.semaphores[k]][0],
                'z': self.model.avenue.positions[self.semaphores[k]][1],
                'dir': math.atan2(self.semaphores[k].direction[0], self.semaphores[k].direction[1]) * 180 / math.pi,
                'state': self.semaphores[k].state
            })
    def step(self):
        self.semaphores.update()

        self.cars.update_position()
        self.cars.update_speed()
        car_list = []
        for k in range(self.p.cars):
            car_list.append({
                'id': k,
                'x': self.model.avenue.positions[self.cars[k]][0],
                'z': self.model.avenue.positions[self.cars[k]][1],
                'dir': math.atan2(self.cars[k].direction[0], self.cars[k].direction[1]) * 180 / math.pi
            })
        sempahore_list = []
        for k in range(len(self.semaphores)):
            sempahore_list.append({
                'id': k + model.p.cars,
                'state': self.semaphores[k].state
            })

        self.data['frames'].append({
            'frame': self.frames,
            'cars': car_list,
            'semaphores': sempahore_list
        })

        self.frames += 1
    def update(self):
        crashes = len(self.cars.select(self.cars.state == 0))
        self.record('Colisiones', crashes)
        
    def end(self):
        crashes = len(self.cars.select(self.cars.state == 0))
        self.report('Colisiones', crashes)
        json_file = json.dumps(self.data)
        with open('simul_data.json', 'w') as outfile:
            outfile.write(json_file)
        
    

In [5]:
def animation_plot_single(m,ax):
    ## generar grafica
    
    ax.set_title(f"Avenida t={m.t*m.p.step_time: .2f}")
    
    colors = ["green", "yellow", "red"]
    
    pos_s1 = m.avenue.positions[m.semaphores[0]]
    ax.scatter(*pos_s1, s=20, c=colors[m.semaphores[0].state])
    
    pos_s1 = m.avenue.positions[m.semaphores[1]]
    ax.scatter(*pos_s1, s=20, c=colors[m.semaphores[1].state])
    
    ax.set_xlim(0, m.avenue.shape[0])
    ax.set_ylim(0, m.avenue.shape[1])
    
    for car in m.cars:
        pos_c = m.avenue.positions[car]
        ax.scatter(*pos_c, s=20, c="black")
        
    ax.set_axis_off()
    ax.set_aspect('equal','box')
    
def animation_plot(m,p):
    #pa animar
        
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    animation = ap.animate(m(p), fig, ax, animation_plot_single)
    return IPython.display.HTML(animation.to_jshtml(fps=20))
    

In [6]:
parameters = {
    'steps' : 20000,
    'step_time' : 0.1,
    'size' : 500,
    'green' : 40,
    'yellow' : 5,
    'red' : 20,
    'cars' : 10,
    'v0' : 0,
}

In [7]:
model = AvenueModel(parameters)
results = model.run()

Completed: 20000 steps
Run time: 0:00:05.788785
Simulation finished


In [8]:
results.variables.AvenueModel

,Colisiones
t,
0,0
1,0
2,0
3,0
4,0
...,...
19996,0
19997,0
19998,0


In [9]:
animation_plot(AvenueModel, parameters)

Animation size has reached 20982293 bytes, exceeding the limit of 20971520.0. If you're sure you want a larger animation embedded, set the animation.embed_limit rc parameter to a larger value (in MB). This and further frames will be dropped.


KeyboardInterrupt: 